In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from math import radians, cos, sin, asin, sqrt
pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
def haversine(lon1, lat1, lon2, lat2):
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371* c
    m = km * 1000
    return m

In [ ]:
def pickUp(month,day,hour):    
    df = pd.read_csv("2020"+month+"/2020"+month+day+"/2020"+month+day+"-"+hour+".csv")
    df = df.sort_values(["VehicleID","timeR"])
    df = df.drop(columns="Unnamed: 0")
    df = df[df.BKK == True]
    df2 = pd.DataFrame()
    UniqueID = df['VehicleID'].unique()
    a = df.groupby(["VehicleID"])
    for i in UniqueID:
        b = a.get_group(i)
        c = b[(b['for_hire_light'].shift(1) == 1) & (b['for_hire_light'] == 0)]
        for i in range(c['lat'].size):
            lat1 = c['lat'].iloc[i]
            lon1 = c['lon'].iloc[i]
            lat2 = b['lat'].iloc[b['lat'].index.get_loc(c.index[i])-1]
            lon2 = b['lon'].iloc[b['lon'].index.get_loc(c.index[i])-1]
            distance = haversine(lon1,lat1,lon2,lat2)
            noloop = 1
            while distance < 10:
                noloop += 1
                try:
                    lat2 = b['lat'].iloc[b['lat'].index.get_loc(c.index[i])-noloop]
                    lon2 = b['lon'].iloc[b['lon'].index.get_loc(c.index[i])-noloop]
                    distance = haversine(lon1,lat1,lon2,lat2)
                finally:    
                    break
            c["timeBefore"] = b["timeR"].iloc[b['timeR'].index.get_loc(c.index[i])-noloop]
            c["distance"] = distance
            d1 = datetime.strptime(c["timeR"].iloc[i],"%H:%M:%S")
            d2 = datetime.strptime(c["timeBefore"].iloc[i],"%H:%M:%S")
            diff = (d1 - d2).total_seconds() / 60
            c["timeDiff"] = diff
            df2 = pd.concat([df2,c.take([i])])
    return df2

# 1->0 = pick / 0->1 = drop
dfs = []
months = ['01']
hours = ['00','01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23']
days = ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31']
for month in months:
    if month == '01':
        for day in days:
            for hour in hours:
                df2 = pickUp(month,day,hour)
                dfs.append(df2)
                print(day+'-'+hour)
            df3 = pd.concat(dfs)
            df3.to_csv("2020"+month+"/PickUp2020"+month+day+".csv")
            dfs = []    